In [11]:
import numpy as np
import pandas as pd
import sidrapy as sidra
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [36]:
def lista_meses(start_date_str: str, qtd_anos: int) -> str:
    start_date = datetime.strptime(start_date_str[:10], "%Y-%m-%d")
    end_date = start_date - relativedelta(years=qtd_anos)

    current = end_date
    result = []

    while current < start_date:
        result.append(current.strftime("%Y%m"))
        current += relativedelta(months=1)

    # Inverter a lista e juntar em uma única string separada por vírgulas
    return ", ".join(reversed(result))

In [62]:
dados_ipca_brasil = sidra.get_table(
	table_code = "7060",             # código da tabela de interesse
	territorial_level = "1",         # nível territorial
	ibge_territorial_code = "all",   # desagregações desse nível
	variable = "63",                 # variável = IPCA - Variação mensal (%)
	classification = '315/7169',
	period =  lista_meses(str(datetime.now()),3)
)
dados_ipca_brasil.columns = dados_ipca_brasil.iloc[0]
dados_ipca_brasil = dados_ipca_brasil[1:].reset_index(drop=True)
dados_ipca_brasil = dados_ipca_brasil.sort_values(by=["Mês (Código)"])
dados_ipca_brasil['Data'] = pd.to_datetime(dados_ipca_brasil['Mês (Código)'].astype(str) + '01', format='%Y%m%d')
dados_ipca_brasil['Valor'] = dados_ipca_brasil['Valor'].str.replace(',', '.').astype(float)
dados_ipca_brasil = dados_ipca_brasil[['Data', 'Valor']].reset_index(drop=True)

In [64]:
dados_ipca_brasil.to_json('teste.json')